In [ ]:
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER,LATITUDE_FORMATTER
import os,errno
import sys
import cartopy.feature as cfeature
import matplotlib.ticker as mticker
import pandas as pd
import datetime as dt
from scipy.ndimage.measurements import label
from math import sin, cos, sqrt, atan2, radians
import geopy.distance
%matplotlib inline

dir2='/thorncroftlab_rit/ahenny/rain/'
dir1='/thorncroftlab_rit/ahenny/rain/US/ghcnd_all/'
dir='/thorncroftlab_rit/ahenny/rain/DISSERTATION_SCRIPTS_RESULTS/'

In [ ]:
ds=xr.open_dataset(dir+'era_5_fall_mslp_neusa.nc')
slp1=ds['msl']
ds1=xr.open_dataset(dir+'era_5_fall_sfc_remnants_neusa.nc')
#this ds1 is just because I had to download a single timestep from December; ignore if not applicable
slp2=ds1['mslp']
slp2['longitude']=slp1.longitude.values
slp1=slp1.sel(latitude=slp2.latitude.values.tolist())
print(slp1)
print(slp2)

In [ ]:
for i in range(41):
    print(1979+i)
    date_start=dt.datetime(1979+i,9,1,6)
    date_range=[date_start+dt.timedelta(hours=6*x) for x in range(91*4-1)]
    date_range=pd.DatetimeIndex(date_range)
    slp_year=slp1.sel(time=date_range)
    print('A')
    slp_final_day=slp2.sel(time=dt.datetime(1979+i,12,1,0))
    print('B')
    slp_mean=(slp_year.sum(dim='time',skipna=True)+slp_final_day)/(91.*4.)
    print('C')
    if i==0:
        slp_annual_means=slp_mean
    else:
        slp_annual_means=xr.concat([slp_annual_means,slp_mean],dim='time')

In [ ]:
#@author: Michael Schramm on GitHub
#This function is derived from code originally posted by Sat Kumar Tomer
#(satkumartomer@gmail.com)
#See also: http://vsp.pnnl.gov/help/Vsample/Design_Trend_Mann_Kendall.htm

from scipy.stats import norm
import scipy.stats as st
def mk_test(x, alpha=0.05):
    n = len(x)

    # calculate S
    s = 0
    for k in range(n-1):
        for j in range(k+1, n):
            s += np.sign(x[j] - x[k])

    # calculate the unique data
    unique_x, tp = np.unique(x, return_counts=True)
    g = len(unique_x)

    # calculate the var(s)
    if n == g:  # there is no tie
        var_s = (n*(n-1)*(2*n+5))/18
    else:  # there are some ties in data
        var_s = (n*(n-1)*(2*n+5) - np.sum(tp*(tp-1)*(2*tp+5)))/18

    if s > 0:
        z = (s - 1)/np.sqrt(var_s)
    elif s < 0:
        z = (s + 1)/np.sqrt(var_s)
    else: # s == 0:
        z = 0

    # calculate the p_value
    p = 2*(1-norm.cdf(abs(z)))  # two tail test
    h = abs(z) > norm.ppf(1-alpha/2)

    if (z < 0) and h:
        trend = 'decreasing'
    elif (z > 0) and h:
        trend = 'increasing'
    else:
        trend = 'no trend'

    return trend, h, p, z

In [ ]:
slp_annual_means=slp_annual_means/100.
print(slp_annual_means.max().values)

In [ ]:
slopes_array=xr.zeros_like(slp1[0,:,:])
sigs_array=xr.zeros_like(slp1[0,:,:])
sigs_array_10=xr.zeros_like(slp1[0,:,:])
for i in range(slp1.latitude.size):
    print(i)
    for j in range(slp1.longitude.size):
        time_series=slp_annual_means[:,i,j].values.tolist()
        linreg=st.linregress(np.arange(1979,2020,1),time_series)
        slope=linreg[0]
        p_value=linreg[3]
        mk_result=mk_test(time_series,alpha=0.05)[0]
        mk_result1=mk_test(time_series,alpha=0.10)[0]
        slopes_array[i,j]=slope
        if mk_result in ['increasing','decreasing']:
            sigs_array[i,j]=1
        if mk_result1 in ['increasing','decreasing']:
            sigs_array_10[i,j]=1

In [ ]:
proj_map = ccrs.PlateCarree()
fig = plt.figure(figsize=(24,8))
ax=plt.subplot(1,1,1,projection=proj_map)

cax=ax.contourf(slp1.longitude,slp1.latitude,slopes_array,transform=ccrs.PlateCarree(),levels=np.arange(-0.07,0.075,0.005),extend='both',cmap=plt.cm.seismic_r)
cbar=plt.colorbar(cax,pad=0,fraction=0.042)
cbar.ax.tick_params(labelsize=22) 
cbar.set_label('hPa/year',fontsize=22,rotation=90,labelpad=12)
cbar.ax.tick_params(labelsize=20)  
ax.coastlines(resolution='10m')
ax.add_feature(cfeature.STATES.with_scale('10m'),alpha=0.3)
ax.add_feature(cfeature.LAKES.with_scale('50m'))
countries = cfeature.NaturalEarthFeature(category='cultural',name='admin_0_boundary_lines_land',scale='50m',facecolor='none')
ax.add_feature(countries)
    
latmin=12
latmax=59
lonmin=-120
lonmax=0
print((lonmin,lonmax))
print((latmin,latmax))
ax.set_xlim(lonmin,lonmax)
ax.set_ylim(latmin,latmax)
ax.set_title('Fall MSLP trends',fontsize=36)

Y=5
X=8
for i in range(int(slopes_array.longitude.size/Y)):
    for j in range(int(slopes_array.latitude.size/Y)):
        if sigs_array[Y*j,Y*i]==1:
            ax.plot(slopes_array.longitude.values[Y*i],slopes_array.latitude.values[Y*j],transform=ccrs.PlateCarree(),marker='o',color='w',mew=1.75,markersize=6,markerfacecolor="None")
        else:
            if sigs_array_10[Y*j,Y*i]==1:
                if slopes_array[Y*j,Y*i]>0:
                    ax.plot(slopes_array.longitude.values[Y*i],slopes_array.latitude.values[Y*j],transform=ccrs.PlateCarree(),marker='o',color='w',mew=0.5,markersize=6,markerfacecolor="None")
                if slopes_array[Y*j,Y*i]<0:
                    ax.plot(slopes_array.longitude.values[Y*i],slopes_array.latitude.values[Y*j],transform=ccrs.PlateCarree(),marker='o',color='w',mew=0.5,markersize=6,markerfacecolor="None")
        
xticks = [-110,-100,-90,-80,-70,-60,-50,-40,-30,-20,-10]
yticks = [5, 15, 25, 35, 45, 55]
ax.tick_params(labelsize=22)

g1=ax.gridlines(crs=ccrs.PlateCarree(),draw_labels=True,linewidth=1.5,color='gray',alpha=0.0,linestyle='--')
cbar.ax.tick_params(labelsize=16) 
g1.xlabel_style={'size':22,'color':'k'}
g1.ylabel_style={'size':22,'color':'k'}
g1.xformatter=LONGITUDE_FORMATTER
g1.yformatter=LATITUDE_FORMATTER
g1.xlocator = mticker.FixedLocator(xticks)
g1.ylocator = mticker.FixedLocator(yticks)
g1.top_labels=False
g1.right_labels=False

#ax.text(lonmax+15,clat,'mm/day',ha='right',va='center',rotation=270,fontsize=14)
plt.show()

In [ ]:
fig.savefig(dir+'neusa_large_scale_composites_trends_1.png')

In [ ]:
ds=xr.open_dataset(dir+'era_5_fall_u_850_neusa.nc')
print(ds)
u1=ds['u']
ds1=xr.open_dataset(dir+'era_5_fall_pl_remnants_neusa.nc')
print(ds1)
u2=ds1['u']
u2['longitude']=u1.longitude.values
u1=u1.sel(latitude=u2.latitude.values.tolist())

dr=xr.open_dataset(dir+'era_5_fall_v_850_neusa.nc')
v1=dr['v']
v2=ds1['v']
v2['longitude']=v1.longitude.values
v1=v1.sel(latitude=v2.latitude.values.tolist())

In [ ]:
for i in range(41):
    print(1979+i)
    date_start=dt.datetime(1979+i,9,1,6)
    date_range=[date_start+dt.timedelta(hours=6*x) for x in range(91*4-1)]
    date_range=pd.DatetimeIndex(date_range)
    u_year=u1.sel(time=date_range)
    u_final_day=u2.sel(time=dt.datetime(1979+i,12,1,0))
    u_mean=(u_year.sum(dim='time',skipna=True)+u_final_day)/(91.*4.)
    
    v_year=v1.sel(time=date_range)
    v_final_day=v2.sel(time=dt.datetime(1979+i,12,1,0))
    v_mean=(v_year.sum(dim='time',skipna=True)+v_final_day)/(91.*4.)
    if i==0:
        u_annual_means=u_mean
        v_annual_means=v_mean
    else:
        u_annual_means=xr.concat([u_annual_means,u_mean],dim='time')
        v_annual_means=xr.concat([v_annual_means,v_mean],dim='time')

In [ ]:
slopes_array_u=xr.zeros_like(slp1[0,:,:])
sigs_array_u=xr.zeros_like(slp1[0,:,:])
sigs_array_u_10=xr.zeros_like(slp1[0,:,:])
slopes_array_v=xr.zeros_like(slp1[0,:,:])
sigs_array_v=xr.zeros_like(slp1[0,:,:])
sigs_array_v_10=xr.zeros_like(slp1[0,:,:])
for i in range(u1.latitude.size):
    print(i)
    for j in range(u1.longitude.size):
        time_seriesu=u_annual_means[:,i,j].values.tolist()
        linreg=st.linregress(np.arange(1979,2020,1),time_seriesu)
        slope=linreg[0]
        p_value=linreg[3]
        mk_resultu=mk_test(time_seriesu,alpha=0.05)[0]
        mk_resultu1=mk_test(time_seriesu,alpha=0.10)[0]
        slopes_array_u[i,j]=slope
        if mk_resultu in ['increasing','decreasing']:
            sigs_array_u[i,j]=1
        if mk_resultu1 in ['increasing','decreasing']:
            sigs_array_u_10[i,j]=1
            
            
        time_seriesv=v_annual_means[:,i,j].values.tolist()
        linreg=st.linregress(np.arange(1979,2020,1),time_seriesv)
        slope=linreg[0]
        p_value=linreg[3]
        mk_resultv=mk_test(time_seriesv,alpha=0.05)[0]
        mk_resultv1=mk_test(time_seriesv,alpha=0.10)[0]
        slopes_array_v[i,j]=slope
        if mk_resultv in ['increasing','decreasing']:
            sigs_array_v[i,j]=1
        if mk_resultv1 in ['increasing','decreasing']:
            sigs_array_v_10[i,j]=1

In [ ]:
proj_map = ccrs.PlateCarree()
fig = plt.figure(figsize=(24,8))
ax=plt.subplot(1,1,1,projection=proj_map)

cax=ax.contourf(u1.longitude,u1.latitude,slopes_array_u,transform=ccrs.PlateCarree(),levels=np.arange(-0.07,0.075,0.005),extend='both',cmap=plt.cm.seismic_r)
cbar=plt.colorbar(cax,pad=0,fraction=0.042)
cbar.ax.tick_params(labelsize=22) 
cbar.set_label('m/s/year',fontsize=22,rotation=90,labelpad=12)
cbar.ax.tick_params(labelsize=20)  
ax.coastlines(resolution='10m')
ax.add_feature(cfeature.STATES.with_scale('10m'),alpha=0.3)
ax.add_feature(cfeature.LAKES.with_scale('50m'))
countries = cfeature.NaturalEarthFeature(category='cultural',name='admin_0_boundary_lines_land',scale='50m',facecolor='none')
ax.add_feature(countries)
    
latmin=12
latmax=59
lonmin=-120
lonmax=0
print((lonmin,lonmax))
print((latmin,latmax))
ax.set_xlim(lonmin,lonmax)
ax.set_ylim(latmin,latmax)
ax.set_title('Fall 850 hPa zonal wind trends',fontsize=36)

Y=5
X=8
for i in range(int(slopes_array.longitude.size/Y)):
    for j in range(int(slopes_array.latitude.size/Y)):
        if sigs_array_u[Y*j,Y*i]==1:
            ax.plot(slopes_array_u.longitude.values[Y*i],slopes_array_u.latitude.values[Y*j],transform=ccrs.PlateCarree(),marker='o',color='w',mew=1.75,markersize=6,markerfacecolor="None")
        else:
            if sigs_array_u_10[Y*j,Y*i]==1:
                if slopes_array[Y*j,Y*i]>0:
                    ax.plot(slopes_array_u.longitude.values[Y*i],slopes_array_u.latitude.values[Y*j],transform=ccrs.PlateCarree(),marker='o',color='w',mew=0.5,markersize=6,markerfacecolor="None")
                if slopes_array[Y*j,Y*i]<0:
                    ax.plot(slopes_array_u.longitude.values[Y*i],slopes_array_u.latitude.values[Y*j],transform=ccrs.PlateCarree(),marker='o',color='w',mew=0.5,markersize=6,markerfacecolor="None")
        
xticks = [-110,-100,-90,-80,-70,-60,-50,-40,-30,-20,-10]
yticks = [5, 15, 25, 35, 45, 55]
ax.tick_params(labelsize=22)

g1=ax.gridlines(crs=ccrs.PlateCarree(),draw_labels=True,linewidth=1.5,color='gray',alpha=0.0,linestyle='--')
cbar.ax.tick_params(labelsize=16) 
g1.xlabel_style={'size':22,'color':'k'}
g1.ylabel_style={'size':22,'color':'k'}
g1.xformatter=LONGITUDE_FORMATTER
g1.yformatter=LATITUDE_FORMATTER
g1.xlocator = mticker.FixedLocator(xticks)
g1.ylocator = mticker.FixedLocator(yticks)
g1.top_labels=False
g1.right_labels=False

#ax.text(lonmax+15,clat,'mm/day',ha='right',va='center',rotation=270,fontsize=14)
plt.show()

In [ ]:
fig.savefig(dir+'neusa_large_scale_composites_trends_2.png')

In [ ]:
proj_map = ccrs.PlateCarree()
fig = plt.figure(figsize=(24,8))
ax=plt.subplot(1,1,1,projection=proj_map)

cax=ax.contourf(u1.longitude,u1.latitude,slopes_array_v,transform=ccrs.PlateCarree(),levels=np.arange(-0.07,0.075,0.005),extend='both',cmap=plt.cm.seismic_r)
cbar=plt.colorbar(cax,pad=0,fraction=0.042)
cbar.ax.tick_params(labelsize=22) 
cbar.set_label('m/s/year',fontsize=22,rotation=90,labelpad=12)
cbar.ax.tick_params(labelsize=20)  
ax.coastlines(resolution='10m')
ax.add_feature(cfeature.STATES.with_scale('10m'),alpha=0.3)
ax.add_feature(cfeature.LAKES.with_scale('50m'))
countries = cfeature.NaturalEarthFeature(category='cultural',name='admin_0_boundary_lines_land',scale='50m',facecolor='none')
ax.add_feature(countries)
    
latmin=12
latmax=59
lonmin=-120
lonmax=0
print((lonmin,lonmax))
print((latmin,latmax))
ax.set_xlim(lonmin,lonmax)
ax.set_ylim(latmin,latmax)
ax.set_title('Fall 850 hPa meridional wind trends',fontsize=36)

Y=5
X=8
for i in range(int(slopes_array.longitude.size/Y)):
    for j in range(int(slopes_array.latitude.size/Y)):
        if sigs_array_v[Y*j,Y*i]==1:
            ax.plot(slopes_array_u.longitude.values[Y*i],slopes_array_u.latitude.values[Y*j],transform=ccrs.PlateCarree(),marker='o',color='w',mew=1.75,markersize=6,markerfacecolor="None")
        else:
            if sigs_array_v_10[Y*j,Y*i]==1:
                if slopes_array[Y*j,Y*i]>0:
                    ax.plot(slopes_array_u.longitude.values[Y*i],slopes_array_u.latitude.values[Y*j],transform=ccrs.PlateCarree(),marker='o',color='w',mew=0.5,markersize=6,markerfacecolor="None")
                if slopes_array[Y*j,Y*i]<0:
                    ax.plot(slopes_array_u.longitude.values[Y*i],slopes_array_u.latitude.values[Y*j],transform=ccrs.PlateCarree(),marker='o',color='w',mew=0.5,markersize=6,markerfacecolor="None")
        
xticks = [-110,-100,-90,-80,-70,-60,-50,-40,-30,-20,-10]
yticks = [5, 15, 25, 35, 45, 55]
ax.tick_params(labelsize=22)

g1=ax.gridlines(crs=ccrs.PlateCarree(),draw_labels=True,linewidth=1.5,color='gray',alpha=0.0,linestyle='--')
cbar.ax.tick_params(labelsize=16) 
g1.xlabel_style={'size':22,'color':'k'}
g1.ylabel_style={'size':22,'color':'k'}
g1.xformatter=LONGITUDE_FORMATTER
g1.yformatter=LATITUDE_FORMATTER
g1.xlocator = mticker.FixedLocator(xticks)
g1.ylocator = mticker.FixedLocator(yticks)
g1.top_labels=False
g1.right_labels=False

#ax.text(lonmax+15,clat,'mm/day',ha='right',va='center',rotation=270,fontsize=14)
plt.show()

In [ ]:
fig.savefig(dir+'neusa_large_scale_composites_trends_3.png')

In [ ]:
proj_map = ccrs.PlateCarree()
fig = plt.figure(figsize=(24,8))
ax=plt.subplot(1,1,1,projection=proj_map)

cax=ax.contourf(slp1.longitude,slp1.latitude,slopes_array,transform=ccrs.PlateCarree(),levels=np.arange(-0.07,0.075,0.005),extend='both',cmap=plt.cm.seismic_r)
cbar=plt.colorbar(cax,pad=0,fraction=0.042)
cbar.ax.tick_params(labelsize=22) 
cbar.set_label('hPa/year',fontsize=22,rotation=90,labelpad=12)
cbar.ax.tick_params(labelsize=20)  
ax.coastlines(resolution='10m')
ax.add_feature(cfeature.STATES.with_scale('10m'),alpha=0.3)
ax.add_feature(cfeature.LAKES.with_scale('50m'))
countries = cfeature.NaturalEarthFeature(category='cultural',name='admin_0_boundary_lines_land',scale='50m',facecolor='none')
ax.add_feature(countries)
    
latmin=12
latmax=59
lonmin=-120
lonmax=0
print((lonmin,lonmax))
print((latmin,latmax))
ax.set_xlim(lonmin,lonmax)
ax.set_ylim(latmin,latmax)
ax.set_title('Fall MSLP and 850 hPa wind trends',fontsize=36)

Y=5
X=5
for i in range(int(slopes_array.longitude.size/Y)):
    for j in range(int(slopes_array.latitude.size/Y)):
        if sigs_array[Y*j,Y*i]==1:
            ax.plot(slopes_array.longitude.values[Y*i],slopes_array.latitude.values[Y*j],transform=ccrs.PlateCarree(),marker='o',color='w',mew=1.75,markersize=6,markerfacecolor="None")
        else:
            if sigs_array_10[Y*j,Y*i]==1:
                if slopes_array[Y*j,Y*i]>0:
                    ax.plot(slopes_array.longitude.values[Y*i],slopes_array.latitude.values[Y*j],transform=ccrs.PlateCarree(),marker='o',color='w',mew=0.5,markersize=6,markerfacecolor="None")
                if slopes_array[Y*j,Y*i]<0:
                    ax.plot(slopes_array.longitude.values[Y*i],slopes_array.latitude.values[Y*j],transform=ccrs.PlateCarree(),marker='o',color='w',mew=0.5,markersize=6,markerfacecolor="None")
        

#for i in range(int(slopes_array.longitude.size/Y)):
#    for j in range(int(slopes_array.latitude.size/Y)):
#        if sigs_array_v[Y*j,Y*i]==1 or sigs_array_u[Y*j,Y*i]==1:
#            print('YES')
#            
#            ax.quiver(u1.longitude[Y*i],u1.longitude[Y*j],slopes_array_u[Y*j,Y*i],slopes_array_v[Y*j,Y*i])
#        else:
#            if sigs_array_v_10[Y*j,Y*i]==1 or sigs_array_u_10[Y*j,Y*i]==1:
#                print('YES')
#                ax.quiver(u1.longitude[Y*i],u1.longitude[Y*j],slopes_array_u[Y*j,Y*i],slopes_array_v[Y*j,Y*i])

sig_uv=sigs_array_u+sigs_array_v
slopes_array_u_select=slopes_array_u.where(sig_uv>0)
slopes_array_v_select=slopes_array_v.where(sig_uv>0)
q=ax.quiver(u1.longitude[0::X].values,u1.latitude[0::X].values,slopes_array_u_select[0::X,0::X].values,slopes_array_v_select[0::X,0::X].values,transform=ccrs.PlateCarree(),units='inches',scale=0.085,width=0.019,alpha=0.85)
ax.quiverkey(q,0.94,1.01,0.1,'0.1 m/s',fontproperties={'size':13})


xticks = [-110,-100,-90,-80,-70,-60,-50,-40,-30,-20,-10]
yticks = [5, 15, 25, 35, 45, 55]
ax.tick_params(labelsize=22)

g1=ax.gridlines(crs=ccrs.PlateCarree(),draw_labels=True,linewidth=1.5,color='gray',alpha=0.0,linestyle='--')
cbar.ax.tick_params(labelsize=16) 
g1.xlabel_style={'size':22,'color':'k'}
g1.ylabel_style={'size':22,'color':'k'}
g1.xformatter=LONGITUDE_FORMATTER
g1.yformatter=LATITUDE_FORMATTER
g1.xlocator = mticker.FixedLocator(xticks)
g1.ylocator = mticker.FixedLocator(yticks)
g1.top_labels=False
g1.right_labels=False

#ax.text(lonmax+15,clat,'mm/day',ha='right',va='center',rotation=270,fontsize=14)
plt.show()

In [ ]:
fig.savefig(dir+'neusa_large_scale_composites_trends_4.png')

In [ ]:
mslp_mean_all=slp_annual_means.mean(dim='time',skipna=True)

In [ ]:
proj_map = ccrs.PlateCarree()
fig = plt.figure(figsize=(24,8))
ax=plt.subplot(1,1,1,projection=proj_map)

cax=ax.contourf(slp1.longitude,slp1.latitude,slopes_array,transform=ccrs.PlateCarree(),levels=np.arange(-0.07,0.075,0.005),extend='both',cmap=plt.cm.seismic_r)
cbar=plt.colorbar(cax,pad=0,fraction=0.042)
cbar.ax.tick_params(labelsize=22) 
cbar.set_label('hPa/year',fontsize=22,rotation=90,labelpad=12)
cbar.ax.tick_params(labelsize=20)  
ax.coastlines(resolution='10m')
ax.add_feature(cfeature.STATES.with_scale('10m'),alpha=0.3)
ax.add_feature(cfeature.LAKES.with_scale('50m'))
countries = cfeature.NaturalEarthFeature(category='cultural',name='admin_0_boundary_lines_land',scale='50m',facecolor='none')
ax.add_feature(countries)
    
latmin=12
latmax=59
lonmin=-120
lonmax=0
print((lonmin,lonmax))
print((latmin,latmax))
ax.set_xlim(lonmin,lonmax)
ax.set_ylim(latmin,latmax)
ax.set_title('Fall MSLP and 850 hPa wind trends',fontsize=36)

Y=5
X=5
for i in range(int(slopes_array.longitude.size/Y)):
    for j in range(int(slopes_array.latitude.size/Y)):
        if sigs_array[Y*j,Y*i]==1:
            ax.plot(slopes_array.longitude.values[Y*i],slopes_array.latitude.values[Y*j],transform=ccrs.PlateCarree(),marker='o',color='w',mew=1.75,markersize=6,markerfacecolor="None")
        else:
            if sigs_array_10[Y*j,Y*i]==1:
                if slopes_array[Y*j,Y*i]>0:
                    ax.plot(slopes_array.longitude.values[Y*i],slopes_array.latitude.values[Y*j],transform=ccrs.PlateCarree(),marker='o',color='w',mew=0.5,markersize=6,markerfacecolor="None")
                if slopes_array[Y*j,Y*i]<0:
                    ax.plot(slopes_array.longitude.values[Y*i],slopes_array.latitude.values[Y*j],transform=ccrs.PlateCarree(),marker='o',color='w',mew=0.5,markersize=6,markerfacecolor="None")
        

#for i in range(int(slopes_array.longitude.size/Y)):
#    for j in range(int(slopes_array.latitude.size/Y)):
#        if sigs_array_v[Y*j,Y*i]==1 or sigs_array_u[Y*j,Y*i]==1:
#            print('YES')
#            
#            ax.quiver(u1.longitude[Y*i],u1.longitude[Y*j],slopes_array_u[Y*j,Y*i],slopes_array_v[Y*j,Y*i])
#        else:
#            if sigs_array_v_10[Y*j,Y*i]==1 or sigs_array_u_10[Y*j,Y*i]==1:
#                print('YES')
#                ax.quiver(u1.longitude[Y*i],u1.longitude[Y*j],slopes_array_u[Y*j,Y*i],slopes_array_v[Y*j,Y*i])

c=ax.contour(mslp_mean_all.longitude,mslp_mean_all.latitude,mslp_mean_all,levels=np.arange(992,1032,2),colors='k',linewidths=0.5,alpha=0.5)
plt.clabel(c,fmt='%.00f',fontsize=15)

sig_uv=sigs_array_u+sigs_array_v
sig_uv10=sigs_array_u_10+sigs_array_v_10
slopes_array_u_select=slopes_array_u.where(sig_uv>0)
slopes_array_v_select=slopes_array_v.where(sig_uv>0)

slopes_array_u_select10=slopes_array_u.where(sig_uv10>0)
slopes_array_v_select10=slopes_array_v.where(sig_uv10>0)

q0=ax.quiver(u1.longitude[0::X].values,u1.latitude[0::X].values,slopes_array_u_select10[0::X,0::X].values,slopes_array_v_select10[0::X,0::X].values,transform=ccrs.PlateCarree(),units='inches',scale=0.085,width=0.019,alpha=0.35)
q=ax.quiver(u1.longitude[0::X].values,u1.latitude[0::X].values,slopes_array_u_select[0::X,0::X].values,slopes_array_v_select[0::X,0::X].values,transform=ccrs.PlateCarree(),units='inches',scale=0.085,width=0.019,alpha=0.85)
ax.quiverkey(q,0.94,1.01,0.1,'0.1 m/s',fontproperties={'size':16})


xticks = [-110,-100,-90,-80,-70,-60,-50,-40,-30,-20,-10]
yticks = [5, 15, 25, 35, 45, 55]
ax.tick_params(labelsize=22)

g1=ax.gridlines(crs=ccrs.PlateCarree(),draw_labels=True,linewidth=1.5,color='gray',alpha=0.0,linestyle='--')
cbar.ax.tick_params(labelsize=16) 
g1.xlabel_style={'size':22,'color':'k'}
g1.ylabel_style={'size':22,'color':'k'}
g1.xformatter=LONGITUDE_FORMATTER
g1.yformatter=LATITUDE_FORMATTER
g1.xlocator = mticker.FixedLocator(xticks)
g1.ylocator = mticker.FixedLocator(yticks)
g1.top_labels=False
g1.right_labels=False

#ax.text(lonmax+15,clat,'mm/day',ha='right',va='center',rotation=270,fontsize=14)
plt.show()

In [ ]:
fig.savefig(dir+'neusa_large_scale_composites_trends_5.png')

In [ ]:
ds=xr.open_dataset(dir+'era_5_fall_ivtx_neusa.nc')
print(ds)
ivtx1=ds['p71.162']
dsa=xr.open_dataset(dir+'era_5_fall_ivty_neusa.nc')
ivty1=dsa['p72.162']

ds1=xr.open_dataset(dir+'era_5_fall_sfc_remnants_neusa.nc')
print(ds1)
ivtx2=ds1['ivtx']
ivtx2['longitude']=ivtx1.longitude.values
ivtx1=ivtx1.sel(latitude=ivtx2.latitude.values.tolist())

ivty2=ds1['ivty']
ivty2['longitude']=ivty1.longitude.values
ivty1=ivty1.sel(latitude=ivty2.latitude.values.tolist())

In [ ]:
for i in range(41):
    print(1979+i)
    date_start=dt.datetime(1979+i,9,1,6)
    date_range=[date_start+dt.timedelta(hours=6*x) for x in range(91*4-1)]
    date_range=pd.DatetimeIndex(date_range)
    
    ivtx_year=ivtx1.sel(time=date_range)
    ivtx_final_day=ivtx2.sel(time=dt.datetime(1979+i,12,1,0))
    ivtx_mean=(ivtx_year.sum(dim='time',skipna=True)+ivtx_final_day)/(91.*4.)
    
    ivty_year=ivty1.sel(time=date_range)
    ivty_final_day=ivty2.sel(time=dt.datetime(1979+i,12,1,0))
    ivty_mean=(ivty_year.sum(dim='time',skipna=True)+ivty_final_day)/(91.*4.)
    if i==0:
        ivtx_annual_means=ivtx_mean
        ivty_annual_means=ivty_mean
    else:
        ivtx_annual_means=xr.concat([ivtx_annual_means,ivtx_mean],dim='time')
        ivty_annual_means=xr.concat([ivty_annual_means,ivty_mean],dim='time')

In [ ]:
slopes_array_ivtx=xr.zeros_like(slp1[0,:,:])
sigs_array_ivtx=xr.zeros_like(slp1[0,:,:])
sigs_array_ivtx_10=xr.zeros_like(slp1[0,:,:])
slopes_array_ivty=xr.zeros_like(slp1[0,:,:])
sigs_array_ivty=xr.zeros_like(slp1[0,:,:])
sigs_array_ivty_10=xr.zeros_like(slp1[0,:,:])
for i in range(ivtx1.latitude.size):
    print(i)
    for j in range(ivtx1.longitude.size):
        time_seriesivtx=ivtx_annual_means[:,i,j].values.tolist()
        linreg=st.linregress(np.arange(1979,2020,1),time_seriesivtx)
        slope=linreg[0]
        p_value=linreg[3]
        mk_resultivtx=mk_test(time_seriesivtx,alpha=0.05)[0]
        mk_resultivtx1=mk_test(time_seriesivtx,alpha=0.10)[0]
        slopes_array_ivtx[i,j]=slope
        if mk_resultivtx in ['increasing','decreasing']:
            sigs_array_ivtx[i,j]=1
        if mk_resultivtx1 in ['increasing','decreasing']:
            sigs_array_ivtx_10[i,j]=1
            
        time_seriesivty=ivty_annual_means[:,i,j].values.tolist()
        linreg=st.linregress(np.arange(1979,2020,1),time_seriesivty)
        slope=linreg[0]
        p_value=linreg[3]
        mk_resultivty=mk_test(time_seriesivty,alpha=0.05)[0]
        mk_resultivty1=mk_test(time_seriesivty,alpha=0.10)[0]
        slopes_array_ivty[i,j]=slope
        if mk_resultivty in ['increasing','decreasing']:
            sigs_array_ivty[i,j]=1
        if mk_resultivty1 in ['increasing','decreasing']:
            sigs_array_ivty_10[i,j]=1

In [ ]:
proj_map = ccrs.PlateCarree()
fig = plt.figure(figsize=(24,8))
ax=plt.subplot(1,1,1,projection=proj_map)

cax=ax.contourf(slp1.longitude,slp1.latitude,slopes_array,transform=ccrs.PlateCarree(),levels=np.arange(-0.07,0.075,0.005),extend='both',cmap=plt.cm.seismic_r)
cbar=plt.colorbar(cax,pad=0,fraction=0.042)
cbar.ax.tick_params(labelsize=22) 
cbar.set_label('hPa/year',fontsize=22,rotation=90,labelpad=12)
cbar.ax.tick_params(labelsize=20)  
ax.coastlines(resolution='10m')
ax.add_feature(cfeature.STATES.with_scale('10m'),alpha=0.3)
ax.add_feature(cfeature.LAKES.with_scale('50m'))
countries = cfeature.NaturalEarthFeature(category='cultural',name='admin_0_boundary_lines_land',scale='50m',facecolor='none')
ax.add_feature(countries)
    
latmin=12
latmax=59
lonmin=-120
lonmax=0
print((lonmin,lonmax))
print((latmin,latmax))
ax.set_xlim(lonmin,lonmax)
ax.set_ylim(latmin,latmax)
ax.set_title('Fall MSLP and IVT trends',fontsize=36)

Y=5
X=5
for i in range(int(slopes_array.longitude.size/Y)):
    for j in range(int(slopes_array.latitude.size/Y)):
        if sigs_array[Y*j,Y*i]==1:
            ax.plot(slopes_array.longitude.values[Y*i],slopes_array.latitude.values[Y*j],transform=ccrs.PlateCarree(),marker='o',color='w',mew=1.75,markersize=6,markerfacecolor="None")
        else:
            if sigs_array_10[Y*j,Y*i]==1:
                if slopes_array[Y*j,Y*i]>0:
                    ax.plot(slopes_array.longitude.values[Y*i],slopes_array.latitude.values[Y*j],transform=ccrs.PlateCarree(),marker='o',color='w',mew=0.5,markersize=6,markerfacecolor="None")
                if slopes_array[Y*j,Y*i]<0:
                    ax.plot(slopes_array.longitude.values[Y*i],slopes_array.latitude.values[Y*j],transform=ccrs.PlateCarree(),marker='o',color='w',mew=0.5,markersize=6,markerfacecolor="None")
        

#for i in range(int(slopes_array.longitude.size/Y)):
#    for j in range(int(slopes_array.latitude.size/Y)):
#        if sigs_array_v[Y*j,Y*i]==1 or sigs_array_u[Y*j,Y*i]==1:
#            print('YES')
#            
#            ax.quiver(u1.longitude[Y*i],u1.longitude[Y*j],slopes_array_u[Y*j,Y*i],slopes_array_v[Y*j,Y*i])
#        else:
#            if sigs_array_v_10[Y*j,Y*i]==1 or sigs_array_u_10[Y*j,Y*i]==1:
#                print('YES')
#                ax.quiver(u1.longitude[Y*i],u1.longitude[Y*j],slopes_array_u[Y*j,Y*i],slopes_array_v[Y*j,Y*i])

c=ax.contour(mslp_mean_all.longitude,mslp_mean_all.latitude,mslp_mean_all,levels=np.arange(992,1032,2),colors='k',linewidths=0.5,alpha=0.5)
plt.clabel(c,fmt='%.00f',fontsize=15)

sig_xy=sigs_array_ivtx+sigs_array_ivty
sig_xy10=sigs_array_ivtx_10+sigs_array_ivty_10
slopes_array_ivtx_select=slopes_array_ivtx.where(sig_xy>0)
slopes_array_ivty_select=slopes_array_ivty.where(sig_xy>0)

slopes_array_ivtx_select10=slopes_array_ivtx.where(sig_xy10>0)
slopes_array_ivty_select10=slopes_array_ivty.where(sig_xy10>0)

q0=ax.quiver(u1.longitude[0::X].values,u1.latitude[0::X].values,slopes_array_ivtx_select10[0::X,0::X].values,slopes_array_ivty_select10[0::X,0::X].values,transform=ccrs.PlateCarree(),units='inches',scale=2.5,width=0.019,alpha=0.35)
q=ax.quiver(u1.longitude[0::X].values,u1.latitude[0::X].values,slopes_array_ivtx_select[0::X,0::X].values,slopes_array_ivty_select[0::X,0::X].values,transform=ccrs.PlateCarree(),units='inches',scale=2.5,width=0.019,alpha=0.85)
ax.quiverkey(q,0.94,1.01,1,'1 kg/m/s',fontproperties={'size':16})


xticks = [-110,-100,-90,-80,-70,-60,-50,-40,-30,-20,-10]
yticks = [5, 15, 25, 35, 45, 55]
ax.tick_params(labelsize=22)

g1=ax.gridlines(crs=ccrs.PlateCarree(),draw_labels=True,linewidth=1.5,color='gray',alpha=0.0,linestyle='--')
cbar.ax.tick_params(labelsize=16) 
g1.xlabel_style={'size':22,'color':'k'}
g1.ylabel_style={'size':22,'color':'k'}
g1.xformatter=LONGITUDE_FORMATTER
g1.yformatter=LATITUDE_FORMATTER
g1.xlocator = mticker.FixedLocator(xticks)
g1.ylocator = mticker.FixedLocator(yticks)
g1.top_labels=False
g1.right_labels=False

#ax.text(lonmax+15,clat,'mm/day',ha='right',va='center',rotation=270,fontsize=14)
plt.show()

In [ ]:
fig.savefig(dir+'neusa_large_scale_composites_trends_6.png')

In [ ]:
#now look at percentage changes in q vs v
wind_slope_mag=np.sqrt(slopes_array_u**2+slopes_array_v**2)
u_all_mean=u_annual_means.mean(dim='time',skipna=True)
v_all_mean=v_annual_means.mean(dim='time',skipna=True)
wind_mag_mean=np.sqrt(u_all_mean**2+v_all_mean**2)
wind_slope_mag_percent=wind_slope_mag/wind_mag_mean*100.

ds=xr.open_dataset(dir+'era_5_fall_q_850_neusa.nc')
q1=ds['q']
ds1=xr.open_dataset(dir+'era_5_fall_pl_remnants_neusa.nc')
q2=ds1['q']
q2['longitude']=q1.longitude.values
q1=q1.sel(latitude=q2.latitude.values.tolist())

In [ ]:
for i in range(41):
    print(1979+i)
    date_start=dt.datetime(1979+i,9,1,6)
    date_range=[date_start+dt.timedelta(hours=6*x) for x in range(91*4-1)]
    date_range=pd.DatetimeIndex(date_range)
    q_year=q1.sel(time=date_range)
    q_final_day=q2.sel(time=dt.datetime(1979+i,12,1,0))
    q_mean=(q_year.sum(dim='time',skipna=True)+q_final_day)/(91.*4.)

    if i==0:
        q_annual_means=q_mean
    else:
        q_annual_means=xr.concat([q_annual_means,q_mean],dim='time')

In [ ]:
slopes_array_q=xr.zeros_like(slp1[0,:,:])
sigs_array_q=xr.zeros_like(slp1[0,:,:])
sigs_array_q_10=xr.zeros_like(slp1[0,:,:])
for i in range(u1.latitude.size):
    print(i)
    for j in range(u1.longitude.size):
        time_seriesq=q_annual_means[:,i,j].values.tolist()
        linreg=st.linregress(np.arange(1979,2020,1),time_seriesq)
        slope=linreg[0]
        p_value=linreg[3]
        mk_resultq=mk_test(time_seriesq,alpha=0.05)[0]
        mk_resultq1=mk_test(time_seriesq,alpha=0.10)[0]
        slopes_array_q[i,j]=slope
        if mk_resultq in ['increasing','decreasing']:
            sigs_array_q[i,j]=1
        if mk_resultq1 in ['increasing','decreasing']:
            sigs_array_q_10[i,j]=1

In [ ]:
q_mean_all=q_annual_means.mean(dim='time',skipna=True)
q_slope_percent=slopes_array_q/q_mean_all*100.

In [ ]:
proj_map = ccrs.PlateCarree()
fig = plt.figure(figsize=(24,8))
ax=plt.subplot(1,1,1,projection=proj_map)

cax=ax.contourf(slp1.longitude,slp1.latitude,q_slope_percent,transform=ccrs.PlateCarree(),levels=np.arange(-1.2,1.3,0.1),extend='both',cmap=plt.cm.seismic_r)
cbar=plt.colorbar(cax,pad=0,fraction=0.042)
cbar.ax.tick_params(labelsize=22) 
cbar.set_label('%/year',fontsize=22,rotation=90,labelpad=12)
cbar.ax.tick_params(labelsize=20)  
ax.coastlines(resolution='10m')
ax.add_feature(cfeature.STATES.with_scale('10m'),alpha=0.3)
ax.add_feature(cfeature.LAKES.with_scale('50m'))
countries = cfeature.NaturalEarthFeature(category='cultural',name='admin_0_boundary_lines_land',scale='50m',facecolor='none')
ax.add_feature(countries)
    
latmin=12
latmax=59
lonmin=-120
lonmax=0
print((lonmin,lonmax))
print((latmin,latmax))
ax.set_xlim(lonmin,lonmax)
ax.set_ylim(latmin,latmax)
ax.set_title('Fall 850 hPa specific humidity trends',fontsize=36)

Y=5
X=5
for i in range(int(slopes_array.longitude.size/Y)):
    for j in range(int(slopes_array.latitude.size/Y)):
        if sigs_array_q[Y*j,Y*i]==1:
            ax.plot(slopes_array.longitude.values[Y*i],slopes_array.latitude.values[Y*j],transform=ccrs.PlateCarree(),marker='o',color='w',mew=1.5,markersize=6,alpha=0.8,markerfacecolor="None")
        else:
            if sigs_array_q_10[Y*j,Y*i]==1:
                if slopes_array_q[Y*j,Y*i]>0:
                    ax.plot(slopes_array.longitude.values[Y*i],slopes_array.latitude.values[Y*j],transform=ccrs.PlateCarree(),marker='o',color='w',mew=0.5,markersize=6,markerfacecolor="None")
                if slopes_array_q[Y*j,Y*i]<0:
                    ax.plot(slopes_array.longitude.values[Y*i],slopes_array.latitude.values[Y*j],transform=ccrs.PlateCarree(),marker='o',color='w',mew=0.5,markersize=6,markerfacecolor="None")
        
#c=ax.contour(mslp_mean_all.longitude,mslp_mean_all.latitude,mslp_mean_all,levels=np.arange(992,1032,2),colors='k',linewidths=0.5,alpha=0.5)
#plt.clabel(c,fmt='%.00f',fontsize=15)

xticks = [-110,-100,-90,-80,-70,-60,-50,-40,-30,-20,-10]
yticks = [5, 15, 25, 35, 45, 55]
ax.tick_params(labelsize=22)

g1=ax.gridlines(crs=ccrs.PlateCarree(),draw_labels=True,linewidth=1.5,color='gray',alpha=0.0,linestyle='--')
cbar.ax.tick_params(labelsize=16) 
g1.xlabel_style={'size':22,'color':'k'}
g1.ylabel_style={'size':22,'color':'k'}
g1.xformatter=LONGITUDE_FORMATTER
g1.yformatter=LATITUDE_FORMATTER
g1.xlocator = mticker.FixedLocator(xticks)
g1.ylocator = mticker.FixedLocator(yticks)
g1.top_labels=False
g1.right_labels=False

#ax.text(lonmax+15,clat,'mm/day',ha='right',va='center',rotation=270,fontsize=14)
plt.show()

In [ ]:
fig.savefig(dir+'neusa_large_scale_composites_trends_7.png')

In [ ]:
speed_annual_means=np.sqrt(u_annual_means**2+v_annual_means**2)
slopes_array_speed=xr.zeros_like(slp1[0,:,:])
sigs_array_speed=xr.zeros_like(slp1[0,:,:])
sigs_array_speed_10=xr.zeros_like(slp1[0,:,:])
for i in range(u1.latitude.size):
    print(i)
    for j in range(u1.longitude.size):
        time_seriesspeed=speed_annual_means[:,i,j].values.tolist()
        linreg=st.linregress(np.arange(1979,2020,1),time_seriesspeed)
        slope=linreg[0]
        p_value=linreg[3]
        mk_resultspeed=mk_test(time_seriesspeed,alpha=0.05)[0]
        mk_resultspeed1=mk_test(time_seriesspeed,alpha=0.10)[0]
        slopes_array_speed[i,j]=slope
        if mk_resultspeed in ['increasing','decreasing']:
            sigs_array_speed[i,j]=1
        if mk_resultspeed1 in ['increasing','decreasing']:
            sigs_array_speed_10[i,j]=1

In [ ]:
speed_mean_all=speed_annual_means.mean(dim='time',skipna=True)
slopes_array_speed_percent=slopes_array_speed/speed_mean_all*100.

proj_map = ccrs.PlateCarree()
fig = plt.figure(figsize=(24,8))
ax=plt.subplot(1,1,1,projection=proj_map)

cax=ax.contourf(slp1.longitude,slp1.latitude,slopes_array_speed_percent,transform=ccrs.PlateCarree(),levels=np.arange(-1.2,1.3,0.1),extend='both',cmap=plt.cm.seismic_r)
cbar=plt.colorbar(cax,pad=0,fraction=0.042)
cbar.ax.tick_params(labelsize=22) 
cbar.set_label('%/year',fontsize=22,rotation=90,labelpad=12)
cbar.ax.tick_params(labelsize=20)  
ax.coastlines(resolution='10m')
ax.add_feature(cfeature.STATES.with_scale('10m'),alpha=0.3)
ax.add_feature(cfeature.LAKES.with_scale('50m'))
countries = cfeature.NaturalEarthFeature(category='cultural',name='admin_0_boundary_lines_land',scale='50m',facecolor='none')
ax.add_feature(countries)
    
latmin=12
latmax=59
lonmin=-120
lonmax=0
print((lonmin,lonmax))
print((latmin,latmax))
ax.set_xlim(lonmin,lonmax)
ax.set_ylim(latmin,latmax)
ax.set_title('Fall 850 hPa wind trends',fontsize=36)

Y=5
X=5

for i in range(int(slopes_array.longitude.size/Y)):
    for j in range(int(slopes_array.latitude.size/Y)):
        if sigs_array_speed[Y*j,Y*i]>0:
            ax.plot(slopes_array.longitude.values[Y*i],slopes_array.latitude.values[Y*j],transform=ccrs.PlateCarree(),marker='o',color='w',mew=1.5,markersize=6,alpha=0.8,markerfacecolor="None")
        else:
            if sigs_array_speed[Y*j,Y*i]>0:
                if slopes_array_speed[Y*j,Y*i]>0:
                    ax.plot(slopes_array.longitude.values[Y*i],slopes_array.latitude.values[Y*j],transform=ccrs.PlateCarree(),marker='o',color='w',mew=0.5,markersize=6,markerfacecolor="None")
                if slopes_array_speed[Y*j,Y*i]<0:
                    ax.plot(slopes_array.longitude.values[Y*i],slopes_array.latitude.values[Y*j],transform=ccrs.PlateCarree(),marker='o',color='w',mew=0.5,markersize=6,markerfacecolor="None")
        
#c=ax.contour(mslp_mean_all.longitude,mslp_mean_all.latitude,mslp_mean_all,levels=np.arange(992,1032,2),colors='k',linewidths=0.5,alpha=0.5)
#plt.clabel(c,fmt='%.00f',fontsize=15)

sig_uv=sigs_array_u+sigs_array_v
slopes_array_u_select=slopes_array_u.where(sig_uv>0)
slopes_array_v_select=slopes_array_v.where(sig_uv>0)
q=ax.quiver(u1.longitude[0::X].values,u1.latitude[0::X].values,slopes_array_u_select[0::X,0::X].values,slopes_array_v_select[0::X,0::X].values,transform=ccrs.PlateCarree(),units='inches',scale=0.085,width=0.019,alpha=0.85,zorder=20)
ax.quiverkey(q,0.94,1.01,0.1,'0.1 m/s',fontproperties={'size':13})

xticks = [-110,-100,-90,-80,-70,-60,-50,-40,-30,-20,-10]
yticks = [5, 15, 25, 35, 45, 55]
ax.tick_params(labelsize=22)

g1=ax.gridlines(crs=ccrs.PlateCarree(),draw_labels=True,linewidth=1.5,color='gray',alpha=0.0,linestyle='--')
cbar.ax.tick_params(labelsize=16) 
g1.xlabel_style={'size':22,'color':'k'}
g1.ylabel_style={'size':22,'color':'k'}
g1.xformatter=LONGITUDE_FORMATTER
g1.yformatter=LATITUDE_FORMATTER
g1.xlocator = mticker.FixedLocator(xticks)
g1.ylocator = mticker.FixedLocator(yticks)
g1.top_labels=False
g1.right_labels=False

#ax.text(lonmax+15,clat,'mm/day',ha='right',va='center',rotation=270,fontsize=14)
plt.show()

In [ ]:
fig.savefig(dir+'neusa_large_scale_composites_trends_8.png')

In [ ]:
ivt_thresh=528.6942100292907
for i in range(41):
    print(1979+i)
    date_start=dt.datetime(1979+i,9,1,6)
    date_range=[date_start+dt.timedelta(hours=6*x) for x in range(91*4-1)]
    date_range=pd.DatetimeIndex(date_range)
    
    ivtx_year=ivtx1.sel(time=date_range)
    ivtx_final_day=ivtx2.sel(time=dt.datetime(1979+i,12,1,0))
    
    ivty_year=ivty1.sel(time=date_range)
    ivty_final_day=ivty2.sel(time=dt.datetime(1979+i,12,1,0))
    
    ivt_mag_year=np.sqrt(ivtx_year**2+ivty_year**2)
    ivt_mag_final_day=np.sqrt(ivtx_final_day**2+ivty_final_day**2)
    ivt_mag_extreme_year=ivt_mag_year.where(ivt_mag_year>=ivt_thresh)
    ivt_mag_extreme_final_day=ivt_mag_final_day.where(ivt_mag_final_day>=ivt_thresh)
    
    eivt_ones_1=ivt_mag_extreme_year/ivt_mag_extreme_year
    eivt_ones_2=ivt_mag_extreme_final_day/ivt_mag_extreme_final_day
    eivt_freq_1=eivt_ones_1.sum(dim='time',skipna=True)
    eivt_freq_2=eivt_ones_2
    
    eivt_freq=eivt_freq_1+eivt_freq_2
    
    if i==0:
        eivt_freq_annual=eivt_freq
    else:
        eivt_freq_annual=xr.concat([eivt_freq_annual,eivt_freq],dim='time')

In [ ]:
ds=xr.open_dataset(dir+'era_5_fall_u_250_neusa.nc')
print(ds)
u1=ds['u']
ds1=xr.open_dataset(dir+'era_5_fall_pl1_remnants_neusa.nc')
print(ds1)
u2=ds1['u']
u2['longitude']=u1.longitude.values
u1=u1.sel(latitude=u2.latitude.values.tolist())

dr=xr.open_dataset(dir+'era_5_fall_v_250_neusa.nc')
v1=dr['v']
v2=ds1['v']
v2['longitude']=v1.longitude.values
v1=v1.sel(latitude=v2.latitude.values.tolist())

In [ ]:
for i in range(41):
    print(1979+i)
    date_start=dt.datetime(1979+i,9,1,6)
    date_range=[date_start+dt.timedelta(hours=6*x) for x in range(91*4-1)]
    date_range=pd.DatetimeIndex(date_range)
    u_year=u1.sel(time=date_range)
    u_final_day=u2.sel(time=dt.datetime(1979+i,12,1,0))
    u_mean=(u_year.sum(dim='time',skipna=True)+u_final_day)/(91.*4.)
    
    v_year=v1.sel(time=date_range)
    v_final_day=v2.sel(time=dt.datetime(1979+i,12,1,0))
    v_mean=(v_year.sum(dim='time',skipna=True)+v_final_day)/(91.*4.)
    if i==0:
        u_annual_means=u_mean
        v_annual_means=v_mean
    else:
        u_annual_means=xr.concat([u_annual_means,u_mean],dim='time')
        v_annual_means=xr.concat([v_annual_means,v_mean],dim='time')

In [ ]:
mag_annual_means=np.sqrt(u_annual_means**2+v_annual_means**2)

In [ ]:
slopes_array_u=xr.zeros_like(slp1[0,:,:])
sigs_array_u=xr.zeros_like(slp1[0,:,:])
sigs_array_u_10=xr.zeros_like(slp1[0,:,:])
slopes_array_v=xr.zeros_like(slp1[0,:,:])
sigs_array_v=xr.zeros_like(slp1[0,:,:])
sigs_array_v_10=xr.zeros_like(slp1[0,:,:])

slopes_array_mag=xr.zeros_like(slp1[0,:,:])
sigs_array_mag=xr.zeros_like(slp1[0,:,:])
sigs_array_mag_10=xr.zeros_like(slp1[0,:,:])
for i in range(u1.latitude.size):
    print(i)
    for j in range(u1.longitude.size):
        time_seriesu=u_annual_means[:,i,j].values.tolist()
        linreg=st.linregress(np.arange(1979,2020,1),time_seriesu)
        slope=linreg[0]
        p_value=linreg[3]
        mk_resultu=mk_test(time_seriesu,alpha=0.05)[0]
        mk_resultu1=mk_test(time_seriesu,alpha=0.10)[0]
        slopes_array_u[i,j]=slope
        if mk_resultu in ['increasing','decreasing']:
            sigs_array_u[i,j]=1
        if mk_resultu1 in ['increasing','decreasing']:
            sigs_array_u_10[i,j]=1
            
            
        time_seriesv=v_annual_means[:,i,j].values.tolist()
        linreg=st.linregress(np.arange(1979,2020,1),time_seriesv)
        slope=linreg[0]
        p_value=linreg[3]
        mk_resultv=mk_test(time_seriesv,alpha=0.05)[0]
        mk_resultv1=mk_test(time_seriesv,alpha=0.10)[0]
        slopes_array_v[i,j]=slope
        if mk_resultv in ['increasing','decreasing']:
            sigs_array_v[i,j]=1
        if mk_resultv1 in ['increasing','decreasing']:
            sigs_array_v_10[i,j]=1
            
        time_seriesmag=mag_annual_means[:,i,j].values.tolist()
        linreg=st.linregress(np.arange(1979,2020,1),time_seriesmag)
        slope=linreg[0]
        p_value=linreg[3]
        mk_resultmag=mk_test(time_seriesmag,alpha=0.05)[0]
        mk_resultmag1=mk_test(time_seriesmag,alpha=0.10)[0]
        slopes_array_mag[i,j]=slope
        if mk_resultmag in ['increasing','decreasing']:
            sigs_array_mag[i,j]=1
        if mk_resultmag1 in ['increasing','decreasing']:
            sigs_array_mag_10[i,j]=1

In [ ]:
mag_mean_all=mag_annual_means.mean(dim='time',skipna=True)

In [ ]:
proj_map = ccrs.PlateCarree()
fig = plt.figure(figsize=(24,8))
ax=plt.subplot(1,1,1,projection=proj_map)

cax=ax.contourf(slp1.longitude,slp1.latitude,slopes_array_mag,transform=ccrs.PlateCarree(),levels=np.arange(-0.15,0.16,0.01),extend='both',cmap=plt.cm.seismic_r)
cbar=plt.colorbar(cax,pad=0,fraction=0.042)
cbar.ax.tick_params(labelsize=22) 
cbar.set_label('m/s/year',fontsize=22,rotation=90,labelpad=12)
cbar.ax.tick_params(labelsize=20)  
ax.coastlines(resolution='10m')
ax.add_feature(cfeature.STATES.with_scale('10m'),alpha=0.3)
ax.add_feature(cfeature.LAKES.with_scale('50m'))
countries = cfeature.NaturalEarthFeature(category='cultural',name='admin_0_boundary_lines_land',scale='50m',facecolor='none')
ax.add_feature(countries)
    
latmin=12
latmax=60
lonmin=-120
lonmax=0
print((lonmin,lonmax))
print((latmin,latmax))
ax.set_xlim(lonmin,lonmax)
ax.set_ylim(latmin,latmax)
ax.set_title('Fall upper-level wind trends',fontsize=36)

Y=5
X=5
for i in range(int(slopes_array.longitude.size/Y)):
    for j in range(int(slopes_array.latitude.size/Y)):
        if sigs_array_mag[Y*j,Y*i]==1:
            ax.plot(slopes_array.longitude.values[Y*i],slopes_array.latitude.values[Y*j],transform=ccrs.PlateCarree(),marker='o',color='w',mew=1.75,markersize=6,markerfacecolor="None")
        else:
            if sigs_array_mag_10[Y*j,Y*i]==1:
                if slopes_array[Y*j,Y*i]>0:
                    ax.plot(slopes_array.longitude.values[Y*i],slopes_array.latitude.values[Y*j],transform=ccrs.PlateCarree(),marker='o',color='w',mew=0.5,markersize=6,markerfacecolor="None")
                if slopes_array[Y*j,Y*i]<0:
                    ax.plot(slopes_array.longitude.values[Y*i],slopes_array.latitude.values[Y*j],transform=ccrs.PlateCarree(),marker='o',color='w',mew=0.5,markersize=6,markerfacecolor="None")
        

#for i in range(int(slopes_array.longitude.size/Y)):
#    for j in range(int(slopes_array.latitude.size/Y)):
#        if sigs_array_v[Y*j,Y*i]==1 or sigs_array_u[Y*j,Y*i]==1:
#            print('YES')
#            
#            ax.quiver(u1.longitude[Y*i],u1.longitude[Y*j],slopes_array_u[Y*j,Y*i],slopes_array_v[Y*j,Y*i])
#        else:
#            if sigs_array_v_10[Y*j,Y*i]==1 or sigs_array_u_10[Y*j,Y*i]==1:
#                print('YES')
#                ax.quiver(u1.longitude[Y*i],u1.longitude[Y*j],slopes_array_u[Y*j,Y*i],slopes_array_v[Y*j,Y*i])

c=ax.contour(mag_mean_all.longitude,mag_mean_all.latitude,mag_mean_all,levels=np.arange(10,40,3),colors='k',linewidths=0.5,alpha=0.5)
plt.clabel(c,fmt='%.00f',fontsize=15)

sig_xy=sigs_array_u+sigs_array_v
sig_xy10=sigs_array_u_10+sigs_array_v_10
slopes_array_u_select=slopes_array_u.where(sig_xy>0)
slopes_array_v_select=slopes_array_v.where(sig_xy>0)

slopes_array_u_select10=slopes_array_u.where(sig_xy10>0)
slopes_array_v_select10=slopes_array_v.where(sig_xy10>0)

q0=ax.quiver(u1.longitude[0::X].values,u1.latitude[0::X].values,slopes_array_u_select10[0::X,0::X].values,slopes_array_v_select10[0::X,0::X].values,transform=ccrs.PlateCarree(),units='inches',scale=0.25,width=0.019,alpha=0.35)
q=ax.quiver(u1.longitude[0::X].values,u1.latitude[0::X].values,slopes_array_u_select[0::X,0::X].values,slopes_array_v_select[0::X,0::X].values,transform=ccrs.PlateCarree(),units='inches',scale=0.25,width=0.019,alpha=0.85)
ax.quiverkey(q,0.94,1.01,0.1,'0.1 m/s/year',fontproperties={'size':16})


xticks = [-110,-100,-90,-80,-70,-60,-50,-40,-30,-20,-10]
yticks = [5, 15, 25, 35, 45, 55]
ax.tick_params(labelsize=22)

g1=ax.gridlines(crs=ccrs.PlateCarree(),draw_labels=True,linewidth=1.5,color='gray',alpha=0.0,linestyle='--')
cbar.ax.tick_params(labelsize=16) 
g1.xlabel_style={'size':22,'color':'k'}
g1.ylabel_style={'size':22,'color':'k'}
g1.xformatter=LONGITUDE_FORMATTER
g1.yformatter=LATITUDE_FORMATTER
g1.xlocator = mticker.FixedLocator(xticks)
g1.ylocator = mticker.FixedLocator(yticks)
g1.top_labels=False
g1.right_labels=False

#ax.text(lonmax+15,clat,'mm/day',ha='right',va='center',rotation=270,fontsize=14)
plt.show()

In [ ]:
fig.savefig(dir+'neusa_large_scale_composites_trends_10.png')